In [21]:
import os 

In [22]:
%pwd

'c:\\Users\\User\\Desktop\\3-IDSD\\mlops\\MLOPS'

In [27]:
os.chdir(r"C:\Users\User\Desktop\3-IDSD\mlops\MLOPS\MlopsProject")

In [28]:
%pwd

'C:\\Users\\User\\Desktop\\3-IDSD\\mlops\\MLOPS\\MlopsProject'

In [32]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [33]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [34]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [35]:
import os
from mlProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [36]:
class DataTransformation:
    def __init__(self, config, target: str):
        self.config = config
        self.target = target
        self.df = pd.read_csv(self.config.data_path)  # Charger le dataset complet

    def handle_missing_target(self):
        self.df.dropna(subset=[self.target], inplace=True)
        if self.df[self.target].dtype == object:
            self.df[self.target] = self.df[self.target].map({'Yes':1,'No':0})
        return self

    def fill_missing_values(self):
        mean_cols = ['MinTemp','MaxTemp','Temp9am','Humidity3pm','Pressure9am','Pressure3pm']
        median_cols = ['Rainfall','WindSpeed9am','WindSpeed3pm','Humidity9am','WindGustSpeed','Sunshine']
        cat_cols = ['WindGustDir', 'WindDir9am', 'WindDir3pm','Location']

        for col in mean_cols:
            if col in self.df.columns: 
                self.df[col] = self.df[col].fillna(self.df[col].mean())
        for col in median_cols:
            if col in self.df.columns: 
                self.df[col] = self.df[col].fillna(self.df[col].median())
        for col in cat_cols:
            if col in self.df.columns: 
                self.df[col] = self.df[col].fillna(self.df[col].mode()[0])

        if 'RainToday' in self.df.columns:
            self.df['RainToday'] = self.df['RainToday'].map({'Yes':1,'No':0}).fillna(0)
        return self


    def log_transform(self):
        skewed_cols = ['Rainfall','WindGustSpeed','Sunshine','RISK_MM']
        for col in skewed_cols:
            if col in self.df.columns: self.df[col] = np.log1p(self.df[col])
        return self

    def handle_outliers(self):
        cols = ['Rainfall','WindGustSpeed','RISK_MM']
        for col in cols:
            if col in self.df.columns:
                upper = self.df[col].quantile(0.99)
                self.df[col] = np.clip(self.df[col], None, upper)
        return self

    def fix_inconsistencies(self):
        if 'MinTemp' in self.df.columns and 'MaxTemp' in self.df.columns:
            invalid = self.df['MinTemp'] > self.df['MaxTemp']
            self.df.loc[invalid, ['MinTemp','MaxTemp']] = self.df.loc[invalid, ['MaxTemp','MinTemp']].values
        if 'RainToday' in self.df.columns:
            self.df.loc[(self.df['Rainfall']>0) & (self.df['RainToday']==0), 'RainToday'] = 1
        return self

    def drop_unnecessary_columns(self):
        drop_cols = ['RISK_MM','Evaporation','Cloud9am','Cloud3pm','Temp3pm']
        for col in drop_cols:
            if col in self.df.columns: self.df.drop(columns=[col], inplace=True)
        return self

    def encode_categorical(self):
        cat_features = ['Location','WindGustDir', 'WindDir9am', 'WindDir3pm']
        for col in cat_features:
            if col in self.df.columns: 
                self.df = pd.get_dummies(self.df, columns=[col], drop_first=True)
        return self

    def extract_weekday(self):
        if 'Date' in self.df.columns:
            self.df['Date'] = pd.to_datetime(self.df['Date'])
            self.df['Weekday'] = self.df['Date'].dt.weekday
            self.df.drop(columns='Date', inplace=True)
        return self

    def transform_and_split(self, test_size=0.20, random_state=42):
        # Applique toutes les transformations
        self.handle_missing_target()\
            .fill_missing_values()\
            .log_transform()\
            .handle_outliers()\
            .fix_inconsistencies()\
            .drop_unnecessary_columns()\
            .encode_categorical()\
            .extract_weekday()

        # Split train/test
        train, test = train_test_split(self.df, test_size=test_size, random_state=random_state)

        # Sauvegarde
        os.makedirs(self.config.root_dir, exist_ok=True)
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info("Data transformed and split into training and test sets")
        logger.info(f"Train shape: {train.shape}, Test shape: {test.shape}")
        print(f"Train shape: {train.shape}, Test shape: {test.shape}")


In [37]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    
    target_column = "RainTomorrow"
    data_transformation = DataTransformation(config=data_transformation_config, target=target_column)
    
    data_transformation.transform_and_split()
except Exception as e:
    raise e


[2025-12-17 20:46:22,096: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-12-17 20:46:22,097: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-17 20:46:22,099: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-12-17 20:46:22,099: INFO: common: created directory at: artifacts]
[2025-12-17 20:46:22,101: INFO: common: created directory at: artifacts/data_transformation]
[2025-12-17 20:46:24,963: INFO: 316921444: Data transformed and split into training and test sets]
[2025-12-17 20:46:24,963: INFO: 316921444: Train shape: (113754, 108), Test shape: (28439, 108)]
Train shape: (113754, 108), Test shape: (28439, 108)


In [38]:
import pandas as pd

train = pd.read_csv("artifacts/data_transformation/train.csv")


In [39]:
train.head()

,MinTemp,MaxTemp,Rainfall,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,...,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW,Weekday
0,15.1,23.9,0.000000,2.251292,4.219508,19.0,22.0,38.0,68.0,1001.9,...,False,False,False,False,False,False,True,False,False,2
1,9.7,14.2,2.151762,2.251292,3.931826,15.0,28.0,91.0,56.0,1008.2,...,False,False,False,False,False,False,True,False,False,0
2,13.2,25.4,0.000000,2.282382,3.433987,6.0,17.0,79.0,63.0,1025.2,...,False,False,False,False,False,False,False,False,False,1
3,7.6,14.8,0.000000,2.079442,4.394449,30.0,35.0,52.0,45.0,1004.6,...,False,False,False,False,False,False,False,True,False,1
4,12.9,22.2,0.000000,2.186051,3.637586,15.0,20.0,69.0,52.0,1023.0,...,False,False,False,True,False,False,False,False,False,0


In [40]:
train.columns

Index(['MinTemp', 'MaxTemp', 'Rainfall', 'Sunshine', 'WindGustSpeed',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am',
       ...
       'WindDir3pm_NW', 'WindDir3pm_S', 'WindDir3pm_SE', 'WindDir3pm_SSE',
       'WindDir3pm_SSW', 'WindDir3pm_SW', 'WindDir3pm_W', 'WindDir3pm_WNW',
       'WindDir3pm_WSW', 'Weekday'],
      dtype='object', length=108)

In [41]:
train.isnull().sum()

MinTemp           0
MaxTemp           0
Rainfall          0
Sunshine          0
WindGustSpeed     0
                 ..
WindDir3pm_SW     0
WindDir3pm_W      0
WindDir3pm_WNW    0
WindDir3pm_WSW    0
Weekday           0
Length: 108, dtype: int64